In [ ]:
import random 
import json 
import os 
import pandas as pd 
from datetime import datetime
import numpy as np


In [55]:
curridr = os.getcwd()
base_dir = os.path.dirname(curridr)

path_file = os.path.join(base_dir,"data")
print(path_file)


file_names = ['books','reading_progress','user_interactions']

/home/ubuntu/project/book-app/analytics/data


In [56]:
dict_fullpath = {}

for file in file_names:
    dict_fullpath[f"{file}"] = {}
    full_path_raw = path_file+f'/raw/{file}.json'
    full_path_processed = path_file+f'/processed/{file}.csv'
    dict_fullpath[f"{file}"]["file_name"] = file
    dict_fullpath[f"{file}"]["full_path_raw"] = full_path_raw
    dict_fullpath[f"{file}"]["full_path_processed"] = full_path_processed

print(dict_fullpath )
    # print(full_path)
    # with open(f"{full_path}") as f:
    #     data = json.load(f)
    #     df = pd.json_normalize(data[f"{file}"])
    #     print(df)

{'books': {'file_name': 'books', 'full_path_raw': '/home/ubuntu/project/book-app/analytics/data/raw/books.json', 'full_path_processed': '/home/ubuntu/project/book-app/analytics/data/processed/books.csv'}, 'reading_progress': {'file_name': 'reading_progress', 'full_path_raw': '/home/ubuntu/project/book-app/analytics/data/raw/reading_progress.json', 'full_path_processed': '/home/ubuntu/project/book-app/analytics/data/processed/reading_progress.csv'}, 'user_interactions': {'file_name': 'user_interactions', 'full_path_raw': '/home/ubuntu/project/book-app/analytics/data/raw/user_interactions.json', 'full_path_processed': '/home/ubuntu/project/book-app/analytics/data/processed/user_interactions.csv'}}


In [68]:
# master data books

with open(f"{dict_fullpath['books']['full_path_raw']}") as f:
    data = json.load(f)
    df = pd.json_normalize(data[f"books"])
    df['id'] = df['id'].astype(int)
    df['rating'] = df['rating'].astype(float)
df.to_csv(f"{dict_fullpath['books']['full_path_processed']}",index=False)


In [71]:
#user_interactions
with open(f"{dict_fullpath['user_interactions']['full_path_raw']}") as f:
    data = json.load(f)
    df = pd.json_normalize(data[f"user_interactions"])
    df['id'] = df['id'].astype(int)
    df['timestamp'] = pd.to_datetime(df["timestamp"])

df.to_csv(f"{dict_fullpath['user_interactions']['full_path_processed']}",index=False)
    


In [70]:
#reading_progress
with open(f"{dict_fullpath['reading_progress']['full_path_raw']}") as f:
    data = json.load(f)
    df = pd.json_normalize(data[f"reading_progress"])
    df['start_read'] = pd.to_datetime(df["start_read"])
    df['end_read'] = pd.to_datetime(df["end_read"])
    df['completion_rate'] = (df['pages_read'] / df['total_pages']).round(2)
    df['checking_pages'] = df['completion_rate']>1 

mean_pages = df["pages_read"].mean()

df["pages_read"] = np.where(df["completion_rate"] > 1, mean_pages, df["pages_read"])

df_clean = df[['user_id', 'book_id','pages_read','total_pages','interaction_id','start_read','end_read','completion_rate']]
df_clean.to_csv(f"{dict_fullpath['reading_progress']['full_path_processed']}",index=False)


In [52]:
df_clean

,user_id,book_id,pages_read,total_pages,interaction_id,start_read,end_read,completion_rate
0,1,6,21.0,309,1,2023-10-30 08:57:13,2023-10-30 11:27:03,0.07
1,10,3,69.0,328,2,2023-02-19 02:01:43,2023-02-19 08:31:03,0.21
2,3,8,419.0,688,3,2023-01-14 08:41:17,2023-01-14 15:24:47,0.61
3,3,1,36.0,180,4,2023-02-15 14:27:49,2023-02-15 15:24:48,0.20
4,8,6,65.0,309,5,2023-08-07 14:20:36,2023-08-07 21:07:33,0.21
...,...,...,...,...,...,...,...,...
95,9,1,141.0,180,96,2023-08-12 20:48:17,2023-08-13 01:49:36,0.78
96,4,3,305.0,328,97,2023-11-14 03:00:12,2023-11-14 03:19:20,0.93
97,3,7,311.0,1216,98,2024-07-13 09:49:09,2024-07-13 15:21:36,0.26
98,5,3,261.0,328,99,2024-10-28 04:05:49,2024-10-28 09:51:43,0.80


In [131]:
dict_max_len_book = {}

for book in data['books']:
    dict_max_len_book[book["id"]]=book['pages']

def book_pages_read():
    dict_book = {}
    rand_pct = random.randint(1,max(dict_max_len_book.keys()))
    rand_pages = random.randint(1,dict_max_len_book[rand_pct])
    dict_book["book_id"] = rand_pct
    dict_book["pages_read"] = rand_pages
    dict_book["total_pages"] = dict_max_len_book[rand_pct]

    return dict_book



{'book_id': 6, 'pages_read': 278, 'total_pages': 309}


In [174]:
rows_to_write = 10 
user_id_max = 10
action = 'view'
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 12, 31)




reading_progres = []
user_interactions = []
for l in range(rows_to_write):
    
    dict_book_read = book_pages_read()
    user_id = random.randint(1,user_id_max)
    random_date = start_date + (end_date - start_date) * random.random()
    random_date_str =  random_date.strftime("%Y-%m-%d %H:%M:%S")

    data_toinsert_reading_progress = {
        "user_id":user_id,
        "book_id":dict_book_read["book_id"],
        "pages_read":dict_book_read["pages_read"],
        "total_pages":dict_book_read["total_pages"],
        "completion_rate":round(int(dict_book_read["pages_read"])/int(dict_book_read["total_pages"]), 2)
    }

    data_toinsert_user_interactions = {
        "user_id":user_id,
        "book_id":dict_book_read["book_id"],
        "actions":action,
        "timestamp":random_date_str
    }


    reading_progres.append(data_toinsert_reading_progress)
    user_interactions.append(data_toinsert_user_interactions)

dict_reading_progres = {}
dict_user_interactions = {}
dict_reading_progres["reading_progress"] = reading_progres
dict_user_interactions["user_interactions"] = user_interactions

print(dict_reading_progres)
print(dict_user_interactions)


{'reading_progress': [{'user_id': 4, 'book_id': 7, 'pages_read': 826, 'total_pages': 1216, 'completion_rate': 0.68}, {'user_id': 9, 'book_id': 4, 'pages_read': 114, 'total_pages': 432, 'completion_rate': 0.26}, {'user_id': 10, 'book_id': 3, 'pages_read': 107, 'total_pages': 328, 'completion_rate': 0.33}, {'user_id': 1, 'book_id': 4, 'pages_read': 27, 'total_pages': 432, 'completion_rate': 0.06}, {'user_id': 1, 'book_id': 3, 'pages_read': 305, 'total_pages': 328, 'completion_rate': 0.93}, {'user_id': 4, 'book_id': 6, 'pages_read': 35, 'total_pages': 309, 'completion_rate': 0.11}, {'user_id': 7, 'book_id': 4, 'pages_read': 241, 'total_pages': 432, 'completion_rate': 0.56}, {'user_id': 3, 'book_id': 8, 'pages_read': 569, 'total_pages': 688, 'completion_rate': 0.83}, {'user_id': 3, 'book_id': 6, 'pages_read': 287, 'total_pages': 309, 'completion_rate': 0.93}, {'user_id': 3, 'book_id': 2, 'pages_read': 232, 'total_pages': 376, 'completion_rate': 0.62}]}
{'user_interactions': [{'user_id': 4,

In [175]:
dict_to_save = {"reading_progress":dict_reading_progres,
                "user_interactions":dict_user_interactions
                }

for li in dict_to_save.items():
    print(li[0])
    print(li[1])
    parent_path_file_reading_progres=os.path.join(base_dir, "data","raw")
    path_to_save = parent_path_file_reading_progres+f'/{li[0]}.json'
    # print(path_to_save)
    with open(path_to_save, "w") as f:
        json.dump(li[1], f, indent=4)

reading_progress
{'reading_progress': [{'user_id': 4, 'book_id': 7, 'pages_read': 826, 'total_pages': 1216, 'completion_rate': 0.68}, {'user_id': 9, 'book_id': 4, 'pages_read': 114, 'total_pages': 432, 'completion_rate': 0.26}, {'user_id': 10, 'book_id': 3, 'pages_read': 107, 'total_pages': 328, 'completion_rate': 0.33}, {'user_id': 1, 'book_id': 4, 'pages_read': 27, 'total_pages': 432, 'completion_rate': 0.06}, {'user_id': 1, 'book_id': 3, 'pages_read': 305, 'total_pages': 328, 'completion_rate': 0.93}, {'user_id': 4, 'book_id': 6, 'pages_read': 35, 'total_pages': 309, 'completion_rate': 0.11}, {'user_id': 7, 'book_id': 4, 'pages_read': 241, 'total_pages': 432, 'completion_rate': 0.56}, {'user_id': 3, 'book_id': 8, 'pages_read': 569, 'total_pages': 688, 'completion_rate': 0.83}, {'user_id': 3, 'book_id': 6, 'pages_read': 287, 'total_pages': 309, 'completion_rate': 0.93}, {'user_id': 3, 'book_id': 2, 'pages_read': 232, 'total_pages': 376, 'completion_rate': 0.62}]}
user_interactions
{'